This notebook animates one of the human-MRP interactions for the study.

In [1]:
import time

import rospy
from geometry_msgs.msg import Twist

import tf
from gazebo_msgs.msg import ModelState 
from gazebo_msgs.srv import GetModelState
from gazebo_msgs.srv import SetModelState

In [2]:
get_state = rospy.ServiceProxy('/gazebo/get_model_state', GetModelState)
set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)

## Helper Functions

In [3]:
def get_model_state_from_response(model_name, model_state_response):
    new_model_state = ModelState()
    new_model_state.model_name = model_name
    
    # copy over post and orientation
    new_model_state.pose = model_state_response.pose
    new_model_state.twist = model_state_response.twist
    
    return new_model_state

In [4]:
def get_world_state():
    models = {
        'tb_sim': None,
        'local_user': None,
        'elevator_door_left': None,
        'elevator_door_right': None
    }

    rospy.wait_for_service('/gazebo/get_model_state')

    try:
        for model_name in models.keys():
            model_msg = get_state(model_name, 'world')
            
            assert model_msg.success is True
            
            models[model_name] = get_model_state_from_response(
                model_name, model_msg)

    except rospy.ServiceException:
        print('/gazebo/get_model_state service call failed')
    
    return models

In [5]:
def reset_world(models):
    rospy.wait_for_service('/gazebo/set_model_state')
    
    try:
        for model_name, model_state in models.items():
            set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
            
            set_state(model_state)
            
    except rospy.ServiceException:
        print ("Service call failed: %s" % str(rospy.ServiceException))

In [6]:
def move_elevator_doors(direction):
    models = get_world_state()
    delta_t = direction * 0.01

    left_door_state = models['elevator_door_left']
    right_door_state = models['elevator_door_right']

    for _ in range(100):
        left_door_state.pose.position.y -= delta_t
        right_door_state.pose.position.y += delta_t

        rospy.wait_for_service('/gazebo/set_model_state')
        try:
            set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
            set_state(left_door_state)
            set_state(right_door_state)

        except rospy.ServiceException:
            print ("Service call failed: %s" % str(rospy.ServiceException))

In [7]:
def get_local_user_state():
    person_state = None
    rospy.wait_for_service('/gazebo/get_model_state')

    try:
        person_state = get_state('local_user', 'world')
        assert person_state.success is True
    except rospy.ServiceException:
        print("/gazebo/get_model_state service call failed")
        
    return get_model_state_from_response('local_user', person_state)

In [8]:
def move_local_user(delta_x, delta_y, delta_z,
                    delta_t, num_iters, current_state, current_theta):
    
    state_msg = current_state
    theta = current_theta
    
    for _ in range(num_iters):
        state_msg.model_name = 'local_user'
        state_msg.pose.position.x += delta_x
        state_msg.pose.position.y += delta_y
        state_msg.pose.position.z += delta_z

        theta += delta_t
        quaternion = tf.transformations.quaternion_from_euler(0, 0, theta)
        state_msg.pose.orientation.x = quaternion[0]
        state_msg.pose.orientation.y = quaternion[1]
        state_msg.pose.orientation.z = quaternion[2]
        state_msg.pose.orientation.w = quaternion[3]

        rospy.wait_for_service('/gazebo/set_model_state')
        try:
            set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
            resp = set_state(state_msg)
        except rospy.ServiceException:
            print ("Service call failed: %s" % str(rospy.ServiceException))

In [9]:
# before starting simulation, save the initial world state
initial_world_state = get_world_state()

In [10]:
rospy.init_node('test_node', anonymous=True)
pub = rospy.Publisher('/tb_cmd_vel', Twist, queue_size=10)

## MRP is far away, MRP enters first

In [13]:
# moves MRP to waiting area for elevator
message = Twist()
message.linear.x = 0.4
pub.publish(message)
time.sleep(25)

# stop
pub.publish(Twist())

# elevator arrives
time.sleep(5)
move_elevator_doors(1)
time.sleep(5)

# move MRP to elevator
message = Twist()
message.linear.x = 0.4
pub.publish(message)
time.sleep(12)

# turn into elevator
message = Twist()
message.linear.x = 0.4
message.angular.z = 0.4
pub.publish(message)
time.sleep(12)

# reposition
message = Twist()
message.angular.z = 0.4
pub.publish(message)
time.sleep(20)

# stop
pub.publish(Twist())

# elevator leaves
time.sleep(3)
move_elevator_doors(-1)

In [18]:
pub.publish(Twist())
time.sleep(3)
move_elevator_doors(-1)

In [12]:
pub.publish(Twist())
reset_world(initial_world_state)

## MRP is far away, local user enters first

In [13]:
# moves MRP to waiting area for elevator
message = Twist()
message.linear.x = 0.4
pub.publish(message)
time.sleep(18)

# stop
pub.publish(Twist())

# elevator arrives
time.sleep(5)
move_elevator_doors(1)
time.sleep(5)

# move local user towards elevator
current_state = get_local_user_state()
move_local_user(-0.005, -0.005, 0, 0, 225, current_state, -0.8)

# turn towards elevator
current_state = get_local_user_state()
move_local_user(-0.005, -0.005, 0, -0.01, 80, current_state, -0.8)

# enter elevator
current_state = get_local_user_state()
move_local_user(-0.005, 0, 0, 0, 230, current_state, -1.5)

# turn around
current_state = get_local_user_state()
move_local_user(0, 0, 0, 0.01, 300, current_state, -1.5)

# elevator leaves
time.sleep(3)
move_elevator_doors(-1)

In [12]:
pub.publish(Twist())
reset_world(initial_world_state)

## local user is far away, MRP enters first

In [16]:
# moves local user to waiting area for elevator
current_state = get_local_user_state()
move_local_user(-0.005, -0.01, 0, 0, 275, current_state, -0.8)

# elevator arrives
time.sleep(5)
move_elevator_doors(1)
time.sleep(5)

# move MRP to elevator
message = Twist()
message.linear.x = 0.4
pub.publish(message)
time.sleep(17)

# turn into elevator
message = Twist()
message.linear.x = 0.4
message.angular.z = 0.4
pub.publish(message)
time.sleep(5)

# reposition
message = Twist()
message.angular.z = 0.4
pub.publish(message)
time.sleep(25)

# stop
pub.publish(Twist())

# elevator leaves
time.sleep(3)
move_elevator_doors(-1)

KeyboardInterrupt: 

In [17]:
pub.publish(Twist())
time.sleep(3)
move_elevator_doors(-1)

In [15]:
pub.publish(Twist())
reset_world(initial_world_state)

## local user is far away, local user enters first

In [11]:
# moves local user to waiting area for elevator
current_state = get_local_user_state()
move_local_user(-0.005, -0.01, 0, 0, 275, current_state, -0.8)

# elevator arrives
time.sleep(5)
move_elevator_doors(1)
time.sleep(5)

# move local user towards elevator
current_state = get_local_user_state()
move_local_user(-0.005, -0.005, 0, 0, 225, current_state, -0.8)

# turn towards elevator
current_state = get_local_user_state()
move_local_user(-0.005, -0.005, 0, -0.01, 80, current_state, -0.8)

# enter elevator
current_state = get_local_user_state()
move_local_user(-0.005, 0, 0, 0, 230, current_state, -1.5)

# turn around
current_state = get_local_user_state()
move_local_user(0, 0, 0, 0.01, 300, current_state, -1.5)

# elevator leaves
time.sleep(5)
move_elevator_doors(-1)

In [25]:
pub.publish(Twist())
reset_world(initial_world_state)